In [1]:
#In this notebook i try Temperature Scaling.
#Load mnist , dont use softmax on last layer
#Find a divider that yields lowest classification erro

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
import pandas as pd

In [3]:
#load data
mnist = tf.keras.datasets.mnist.load_data(path="mnist.npz")

In [4]:
print(mnist[0][0].shape)
print(mnist[0][1].shape)
print(mnist[1][0].shape)
print(mnist[1][1].shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
#will not use cnn , will just use MLP
X = mnist[0][0].reshape(mnist[0][0].shape[0],28*28)
X_test = mnist[1][0].reshape(mnist[1][0].shape[0],28*28)
print(X.shape)
print(X_test.shape)
Y = mnist[0][1]
print(Y.shape)

(60000, 784)
(10000, 784)
(60000,)


In [6]:
# Model 
input_layer = keras.layers.Input(shape=(28*28,))
dense_1 = keras.layers.Dense(128, activation='relu')(input_layer)
dense_2 = keras.layers.Dense(128, activation='relu')(dense_1)
logits = keras.layers.Dense(10)(dense_2) # Here 3 represents number of classes
model = keras.Model(inputs=input_layer, outputs=logits)

In [7]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Compile 
custom_loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=custom_loss,
              metrics=['accuracy']
             )

In [9]:
# Fit 
history = model.fit(X, keras.utils.to_categorical(Y),
                    epochs=5,
                    batch_size=8)

Epoch 1/5
7500/7500 [==============================] - 11s 2ms/step - loss: 1.1775 - accuracy: 0.8778
Epoch 2/5
7500/7500 [==============================] - 11s 1ms/step - loss: 0.2706 - accuracy: 0.9307
Epoch 3/5
7500/7500 [==============================] - 11s 1ms/step - loss: 0.2132 - accuracy: 0.9485
Epoch 4/5
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1942 - accuracy: 0.9538
Epoch 5/5
7500/7500 [==============================] - 11s 1ms/step - loss: 0.1830 - accuracy: 0.9571


In [10]:
# Logit prediction
y_pred = model.predict(X)

In [11]:
#just bare final layer numbers, not softmax applied
y_pred[0]

array([  -6.7536936,  -26.910238 ,  -60.678905 ,   46.090233 ,
       -139.72382  ,   66.62791  ,    3.4204516,    2.7258708,
         36.280197 ,   36.544228 ], dtype=float32)

In [12]:
# ECE result before calibration
num_bins = 50
labels_true = tf.convert_to_tensor(Y, dtype=tf.int32, name='labels_true')
logits = tf.convert_to_tensor(y_pred, dtype=tf.float32, name='logits')

In [13]:
labels_true

<tf.Tensor: shape=(60000,), dtype=int32, numpy=array([5, 0, 4, ..., 5, 6, 8], dtype=int32)>

In [14]:
logits[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([  -6.7536936,  -26.910238 ,  -60.678905 ,   46.090233 ,
       -139.72382  ,   66.62791  ,    3.4204516,    2.7258708,
         36.280197 ,   36.544228 ], dtype=float32)>

In [15]:
tfp.stats.expected_calibration_error(num_bins=num_bins, 
                                     logits=logits, 
                                     labels_true=labels_true)

<tf.Tensor: shape=(), dtype=float32, numpy=0.009657455>

In [16]:

# Temperature Scaling

temp = tf.Variable(initial_value=1.0, trainable=True, dtype=tf.float32) 

def compute_loss():
    y_pred_model_w_temp = tf.math.divide(y_pred, temp)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                tf.convert_to_tensor(keras.utils.to_categorical(Y)), y_pred_model_w_temp))
    return loss

optimizer = tf.optimizers.Adam(learning_rate=0.01)

print('Temperature Initial value: ', temp.numpy())

for i in range(300):
    opts = optimizer.minimize(compute_loss, var_list=[temp])


print('Temperature Final value: ',temp.numpy())


Temperature Initial value:  1.0
Temperature Final value:  1.2383403


In [17]:
#when values coincide with indexes ( selected indexes have highest position)
test_labels = [[0.0,0.0,1.0],[0.0,1.0,0.0],[1.0,0.0,0.0]]
test_points = [[7.0,8.0,9.0],[3.0,6.0,5.0],[10.0,4.0,8.0]]
cross_ent_with_logits = tf.nn.softmax_cross_entropy_with_logits(test_labels,test_points)
print("cross_ent_with_logits ",cross_ent_with_logits)
loss = tf.reduce_mean( cross_ent_with_logits ) 
print("loss ",loss)
#loss is 0.29524234

cross_ent_with_logits  tf.Tensor([0.40760595 0.34901217 0.12910892], shape=(3,), dtype=float32)
loss  tf.Tensor(0.29524234, shape=(), dtype=float32)


In [18]:
#when values NOT coincide with indexes ( selected indexes have highest position)
#as seen loss is much more when indexes of real labels do not have highest point
test_labels = [[0.0,0.0,1.0],[0.0,1.0,0.0],[1.0,0.0,0.0]]
test_points = [[17.0,8.0,9.0],[13.0,6.0,5.0],[10.0,14.0,8.0]]
cross_ent_with_logits = tf.nn.softmax_cross_entropy_with_logits(test_labels,test_points)
print("cross_ent_with_logits ",cross_ent_with_logits)
loss = tf.reduce_mean( cross_ent_with_logits ) 
print("loss ",loss)
#loss is 6.3407617

cross_ent_with_logits  tf.Tensor([8.000459  7.0012465 4.0205812], shape=(3,), dtype=float32)
loss  tf.Tensor(6.3407617, shape=(), dtype=float32)


In [19]:
# ECE result after calibration
y_pred_model_w_temp = tf.math.divide(y_pred, temp)
num_bins = 50
labels_true = tf.convert_to_tensor(Y, dtype=tf.int32, name='labels_true')
logits = tf.convert_to_tensor(y_pred_model_w_temp, dtype=tf.float32, name='logits')

tfp.stats.expected_calibration_error(num_bins=num_bins, 
                                     logits=logits, 
                                     labels_true=labels_true)

<tf.Tensor: shape=(), dtype=float32, numpy=0.010399372>

In [20]:
temperature_scaled =  y_pred_model_w_temp[0]
actual_prediction = y_pred[0]
print("temperature_scaled",temperature_scaled)
print("actual_prediction",actual_prediction)

temperature_scaled tf.Tensor(
[  -5.453827   -21.730892   -49.000187    37.21936   -112.83152
   53.8042       2.762126     2.2012293   29.297438    29.51065  ], shape=(10,), dtype=float32)
actual_prediction [  -6.7536936  -26.910238   -60.678905    46.090233  -139.72382
   66.62791      3.4204516    2.7258708   36.280197    36.544228 ]


In [21]:
temperature_multiplied = temperature_scaled * temp.numpy()
#compare numbers with above numbers
print( temperature_multiplied )

tf.Tensor(
[  -6.7536936  -26.910238   -60.678905    46.090233  -139.72382
   66.62791      3.4204519    2.7258708   36.280197    36.544228 ], shape=(10,), dtype=float32)
